In [8]:
import os
import zipfile
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from io import BytesIO
from datetime import datetime

def download_and_unzip(zip_url, extract_to):
    """Download ZIP file from a URL and extract it to the specified folder."""
    response = requests.get(zip_url)
    if response.status_code == 200:
        with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Downloaded and extracted files to: {extract_to}")
    else:
        raise Exception(f"Failed to download. Status code: {response.status_code}")

def parse_xml_to_dataframe(xml_file):
    """Parse XML file and convert it to a DataFrame."""
    tree = ET.parse(xml_file)
    root = tree.getroot()

    all_rows = []
    all_columns = set()  # Track all unique column names

    # Extract data row by row
    for row in root:
        data = {elem.tag: elem.text for elem in row}
        all_columns.update(data.keys())  # Add new columns found
        all_rows.append(data)

    # Convert rows to DataFrame, filling missing columns with NaN
    df = pd.DataFrame(all_rows).reindex(columns=sorted(all_columns))
    return df

def convert_xml_to_csv(xml_folder, output_folder):
    """Convert all XML files in the specified folder to CSV."""
    os.makedirs(output_folder, exist_ok=True)

    for file_name in os.listdir(xml_folder):
        if file_name.endswith('.xml'):
            xml_path = os.path.join(xml_folder, file_name)
            df = parse_xml_to_dataframe(xml_path)

            csv_file = os.path.join(output_folder, file_name.replace('.xml', '.csv'))
            df.to_csv(csv_file, index=False)
            print(f"Converted {file_name} to {csv_file}")

## Code for extracting the files on a daily basis

#url_base='https://prod-grants-gov-chatbot.s3.amazonaws.com/extracts'
#today=datetime.now()
#zip_url=construct_zip_url(url_base,today)

## Example Usage

zip_url = 'https://prod-grants-gov-chatbot.s3.amazonaws.com/extracts/GrantsDBExtract20241019v2.zip'
temp_extract_folder = 'temp_extract'  # Temporary folder for extraction
output_csv_folder = 'tribal_grants_2024_10_12'  # Folder to store CSV files
##output_csv_folder = 'tribal_grants_' + today.strftime('%Y_%m_%d')

# Download, extract, and convert XML files to CSV
download_and_unzip(zip_url, temp_extract_folder)
convert_xml_to_csv(temp_extract_folder, output_csv_folder)

Downloaded and extracted files to: temp_extract
Converted GrantsDBExtract20241019v2.xml to tribal_grants_2024_10_12/GrantsDBExtract20241019v2.csv


In [51]:
import pandas as pd
import matplotlib.pyplot as plt

tribal_df=pd.read_csv('/content/tribal_grants_2024_10_12/GrantsDBExtract20241019v2.csv')
tribal_df.info()

<ipython-input-51-016e3b7d284e>:3: DtypeWarning: Columns (19,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  tribal_df=pd.read_csv('/content/tribal_grants_2024_10_12/GrantsDBExtract20241019v2.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79408 entries, 0 to 79407
Data columns (total 38 columns):
 #   Column                                                                                        Non-Null Count  Dtype  
---  ------                                                                                        --------------  -----  
 0   {http://apply.grants.gov/system/OpportunityDetail-V1.0}AdditionalInformationOnEligibility     55886 non-null  object 
 1   {http://apply.grants.gov/system/OpportunityDetail-V1.0}AdditionalInformationText              51938 non-null  object 
 2   {http://apply.grants.gov/system/OpportunityDetail-V1.0}AdditionalInformationURL               52053 non-null  object 
 3   {http://apply.grants.gov/system/OpportunityDetail-V1.0}AgencyCode                             79365 non-null  object 
 4   {http://apply.grants.gov/system/OpportunityDetail-V1.0}AgencyName                             79364 non-null  object 
 5   {http://apply.grants.gov/

Creating data_source column from the URL

In [54]:
from urllib.parse import urlparse

# Function to safely extract domains from URLs or keep non-URL values as-is
def extract_domain(value):
    # Check if the value starts with 'http://' or 'https://'
    if isinstance(value, str) and (value.startswith('http://') or value.startswith('https://')):
        parsed_url = urlparse(value)  # Parse the URL
        domain = parsed_url.netloc  # Extract domain (netloc)

        # Ensure the domain has at least two parts to avoid IndexError
        parts = domain.split('.')
        if len(parts) >= 2:
            return parts[-2] + '.' + parts[-1]  # Return main domain (e.g., 'grants.gov')
        else:
            return domain  # Return as-is if it's a malformed domain
    else:
        return value  # Keep non-URL values as they are

# Apply the function and create the 'data_source' column
tribal_df['data_source'] = tribal_df['{http://apply.grants.gov/system/OpportunityDetail-V1.0}AdditionalInformationURL'].apply(extract_domain)

# Check the result
tribal_df['data_source'].value_counts()

,count
data_source,
nih.gov,12454
grants.gov,9796
grantsolutions.gov,2657
usda.gov,1675
nasaprs.com,1559
...,...
CLICK ON THE FULL ANNOUNCEMENT BUTTON LOCATED AT THE TOP OF THIS PAGE,1
Full anouncement is available on the Grants.gov website by scrolling to the tope of the page and clicking on the &quot;Full Announcement&quot; box.,1
osha.gov,1


In [57]:
tribal_df=tribal_df.rename(columns={'{http://apply.grants.gov/system/OpportunityDetail-V1.0}OpportunityID':'Grant ID','data_source':'Source of data','{http://apply.grants.gov/system/OpportunityDetail-V1.0}AdditionalInformationURL':'Link to grant source','{http://apply.grants.gov/system/OpportunityDetail-V1.0}AgencyName':'Granting organization','{http://apply.grants.gov/system/OpportunityDetail-V1.0}OpportunityTitle':'Grant Title'})
tribal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79408 entries, 0 to 79407
Data columns (total 39 columns):
 #   Column                                                                                        Non-Null Count  Dtype  
---  ------                                                                                        --------------  -----  
 0   {http://apply.grants.gov/system/OpportunityDetail-V1.0}AdditionalInformationOnEligibility     55886 non-null  object 
 1   {http://apply.grants.gov/system/OpportunityDetail-V1.0}AdditionalInformationText              51938 non-null  object 
 2   Link to grant source                                                                          52053 non-null  object 
 3   {http://apply.grants.gov/system/OpportunityDetail-V1.0}AgencyCode                             79365 non-null  object 
 4   Granting organization                                                                         79364 non-null  object 
 5   {http://apply.grants.gov/

Creating a new df for given meta data format

In [67]:
tribal_df_meta=tribal_df[['Grant Title','Granting organization','Link to grant source','Source of data','Grant ID']]
tribal_df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79408 entries, 0 to 79407
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Grant Title            79408 non-null  object
 1   Granting organization  79364 non-null  object
 2   Link to grant source   52053 non-null  object
 3   Source of data         52053 non-null  object
 4   Grant ID               79408 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


In [72]:
from urllib.parse import urlparse

def extract_website_name(link):
    if pd.isna(link) or link.strip() == "":
        return None
    parsed_url = urlparse(link)
    # Check if the netloc has a valid structure
    if not parsed_url.netloc or '.' not in parsed_url.netloc:
        return None
    domain_parts = parsed_url.netloc.split('.')
    # Return the last two parts as the website name (e.g., 'grants.gov')
    return '.'.join(domain_parts[-2:]) if len(domain_parts) >= 2 else None

# Create a new column for website names safely
tribal_df_meta.loc[:, 'Website'] = tribal_df_meta['Link to grant source'].apply(extract_website_name)

# Fill missing values in 'Source of data' using the extracted website names safely
tribal_df_meta['Source of data'] = tribal_df_meta['Source of data'].fillna(tribal_df_meta['Website'])

# Drop the 'Website' column if you no longer need it
tribal_df_meta.drop(columns=['Website'], inplace=True)

# Check the result
print(tribal_df_meta)

# Check for remaining missing links
remaining_missing_links = tribal_df_meta['Link to grant source'].isna().sum()
print(f'Remaining missing links: {remaining_missing_links}')

                                             Grant Title  \
0      Establishment of the Edmund S. Muskie Graduate...   
1      Eradication of Yellow Crazy Ants on Johnston A...   
2      Cooperative Ecosystem Studies Unit, Piedmont S...   
3      Plant Feedstock Genomics for Bioenergy:  A Joi...   
4      Management of HIV-Related Lung Disease and Car...   
...                                                  ...   
79403  USAID/Sri Lanka Public Financial Management Ac...   
79404  EHB-Benchmark Plan Modernization Grant for Sta...   
79405        Spotlight on Humanities in Higher Education   
79406  Child Welfare Research Partnerships: Understan...   
79407  Request for Information (RFI)-The Enhancing Tr...   

                                 Granting organization  \
0            Bureau of South and Central Asian Affairs   
1                            Fish and Wildlife Service   
2                                    Geological Survey   
3                                    Office of 

<ipython-input-72-1f93a9699097>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tribal_df_meta.loc[:, 'Website'] = tribal_df_meta['Link to grant source'].apply(extract_website_name)
<ipython-input-72-1f93a9699097>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tribal_df_meta['Source of data'] = tribal_df_meta['Source of data'].fillna(tribal_df_meta['Website'])
<ipython-input-72-1f93a9699097>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

In [73]:
tribal_df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79408 entries, 0 to 79407
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Grant Title            79408 non-null  object
 1   Granting organization  79364 non-null  object
 2   Link to grant source   79408 non-null  object
 3   Source of data         79408 non-null  object
 4   Grant ID               79408 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


In [75]:
tribal_df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79408 entries, 0 to 79407
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Grant Title            79408 non-null  object
 1   Granting organization  79364 non-null  object
 2   Link to grant source   79408 non-null  object
 3   Source of data         79408 non-null  object
 4   Grant ID               79408 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


Converting all the links to grants.gov format using Grant ID to ensure all the links are in a common format.

In [96]:
# Function to convert all links to the same Grants.gov format
def convert_all_links_to_grants_gov_format(df, column_name):
    base_url = "http://www.grants.gov/search-results-detail"  # Base URL for Grants.gov
    uniform_links = []

    for index, row in df.iterrows():
        link = row[column_name].strip()  # Strip any whitespace

        # Check if the link already ends with the correct grant ID format
        if link.startswith('http'):
            # Extract the grant ID if the link is valid but not in the required format
            if 'search-results-detail' in link:
                # The link is already in the right format, keep it as is
                uniform_links.append(link)
            else:
                # If the link doesn't match the required format, extract grant ID
                grant_id = link.split('/')[-1]  # Assuming the grant ID is the last part of the URL
                new_link = f"{base_url}/{grant_id}"  # Fixed URL construction
                uniform_links.append(new_link)
        else:
            # If the link is just a grant ID, create the new link
            grant_id = link  # Assuming link is just the grant ID
            new_link = f"{base_url}/{grant_id}"  # Fixed URL construction
            uniform_links.append(new_link)

    # Update the DataFrame with new links using .loc to avoid SettingWithCopyWarning
    df.loc[:, column_name] = uniform_links
    return df

# Convert all links in the DataFrame to the uniform Grants.gov format
tribal_df_meta = convert_all_links_to_grants_gov_format(tribal_df_meta, 'Link to grant source')

# Display the updated DataFrame with converted links
print(tribal_df_meta[['Link to grant source']])


                                    Link to grant source
0      https://www.grants.gov/search-results-detail/2...
1      http://www.grants.gov/search-results-detail/26...
2      http://www.grants.gov/search-results-detail/13...
3      http://www.grants.gov/search-results-detail/ww...
4      http://www.grants.gov/search-results-detail/RF...
...                                                  ...
79403  https://www.grants.gov/search-results-detail/3...
79404  https://www.grants.gov/search-results-detail/3...
79405  http://www.grants.gov/search-results-detail/sp...
79406  https://www.grants.gov/search-results-detail/3...
79407  https://www.grants.gov/search-results-detail/3...

[79408 rows x 1 columns]


In [97]:
tribal_df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79408 entries, 0 to 79407
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Grant Title            79408 non-null  object
 1   Granting organization  79364 non-null  object
 2   Link to grant source   79408 non-null  object
 3   Source of data         79408 non-null  object
 4   Grant ID               79408 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


In [106]:
tribal_df_meta['Applicant Eligibility']=tribal_df['{http://apply.grants.gov/system/OpportunityDetail-V1.0}EligibleApplicants']

<ipython-input-106-a818a03217af>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tribal_df_meta['Applicant Eligibility']=tribal_df['{http://apply.grants.gov/system/OpportunityDetail-V1.0}EligibleApplicants']


In [110]:
tribal_df_meta.to_csv('new_tribal_meta_cleaned_20241019.csv')